<a href="https://colab.research.google.com/github/anustupdas/GoogleCollab_NLP/blob/main/MultiLingual_Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Installation Setup

In [ ]:
!pip install pydub
!pip install -U openai-whisper
!pip install setuptools-rust
!sudo apt update && sudo apt install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

Sun Jun 11 18:33:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Library Functions

In [ ]:
def get_chunks(audio, audio_file, sub_audio_folder, segment_duration_sec = 5):

  audio_duration = audio.duration_seconds * 1000
  initial_start_min = 0
  initial_start_sec = 1
  initial_end_min = 0
  initial_end_sec = segment_duration_sec 
  count = 0
  
# pydub does things in milliseconds, so convert time
  start = ((initial_start_min*60)+initial_start_sec)*1000
  end = ((initial_end_min*60)+initial_end_sec)*1000

  while audio_duration > end:
    count += 1
    # song clip of 10 seconds from starting
    audio_segments = audio[start: end]
    audio_file_final = str(count)+".mp3"
     # save file
    audio_segment_file_path = os.path.join(sub_audio_folder,audio_file_final) 
    audio_segments.export(audio_segment_file_path, format="mp3")
    print(f" Audio file {audio_file} is created and saved in {audio_segment_file_path}")

    if (audio_duration - end) / 1000 < 1:
      break
    else:
      start = end + 10
      end = end + (segment_duration_sec * 1000)


In [ ]:
import whisper
import re

def decode_audio_to_text_adv(sub_audio_folder):

  audio_sub_segments = os.listdir(sub_audio_folder)
  audio_sub_segments.sort(key=lambda f: int(re.sub('\D', '', f)))
  model_en = whisper.load_model("medium.en")
  model = whisper.load_model("base")
  final_text = " "

  print(audio_sub_segments)
  for audio_segment in audio_sub_segments:
    if audio_segment[-4:] != ".mp3":
      continue
    audio_subsegment_file_path =  os.path.join(sub_audio_folder,audio_segment)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_subsegment_file_path)
    audio = whisper.pad_or_trim(audio)
    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    if max(probs, key=probs.get) == 'en':
            result = model_en.transcribe(audio_subsegment_file_path)
    else:
      whisper_bn = pipeline('automatic-speech-recognition', model ="Rakib/whisper-small-bn-all", device = 0)
      result = whisper_bn(audio_subsegment_file_path)
    print(f"For Audio Segment {audio_segment}")
    print(result["text"])
    final_text += result["text"] + " "
    print()
    
    
  return final_text

# **Run Speech To Text**

In [ ]:
from IPython.lib.display import isdir
from pydub import AudioSegment
import json
import os
import glob
import whisper
from google.colab import files

if os.path.isdir("/content/audio") is False:
  os.mkdir("/content/audio")
%cd /content/audio

uploaded = files.upload()

audio_path = "/content/audio/"

audio_files = os.listdir(audio_path)

all_result = {}

for audio_file in audio_files:
  result = {}
  if audio_file[-4:] != ".mp3":
    continue
  audio_file_path = os.path.join(audio_path,audio_file)
  print(audio_file_path) 
  audio = AudioSegment.from_mp3(audio_file_path)

  print("Duration in milliseconds: ", audio.duration_seconds * 1000)
  sub_audio_folder = os.path.join(audio_path,audio_file[:-4])
  if os.path.isdir(sub_audio_folder) is False:
    os.mkdir( os.path.join(audio_path,audio_file[:-4]))
  
  get_chunks(audio, audio_file[:-4], sub_audio_folder, 15)
  extracted_text = decode_audio_to_text_adv(sub_audio_folder)

  sub_audio_folder_b = sub_audio_folder + "/*"
  files = glob.glob(sub_audio_folder_b)
  for f in files:
    os.remove(f)

  transcript_file = os.path.join(sub_audio_folder,audio_file[:-4])
  transcript_file = transcript_file + "_speech.json"
  print(" ")
  print(extracted_text)

  result[audio_file] = extracted_text
  all_result[audio_file] = extracted_text
  try:
    with open(transcript_file, 'w') as fp:
      json.dump(result, fp)
  except:
    print("fucked encoding")
  



/content/audio


/content/audio/Advocate_1.mp3
Duration in milliseconds:  2874496.0
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/1.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/2.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/3.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/4.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/5.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/6.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/7.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/8.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/9.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/10.mp3
 Audio file Advocate_1 is created and saved in /content/audio/Advocate_1/11.mp3
 Audio file Advocate_1 is created and saved in /content/audio/

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


For Audio Segment 50.mp3
আই অবস্ট ফর দিফার্কটি কমপ্লেনেন দ্য গোয়ালাইমার টেকমার নিয়ে নিকল্যান্ট। আই দ্য ফরস্ট ডেট অফিরেন আই স্টুট হেতমাই ফর দিফার্কটি কমপ্লেন্ট একান দ্য বিউল্ভাস রিজেক।

Detected language: en
For Audio Segment 51.mp3
 And there was a date given by the judge on three, four days for police custody and after three, four days of police custody, the next day it was there. On that date...

Detected language: en
For Audio Segment 52.mp3
 Senior Advocate standing on behalf of that accused person Who has high level political post in West Bengal

Detected language: en
For Audio Segment 53.mp3
 So, when I was standing for the de facto and again the bail was rejected, he called me through his PA and he said,

Detected language: en
For Audio Segment 54.mp3
 ask me to step down from the matter ask me whatever money I want he will give me but I should step down from the matter so if this is the scenario involved

Detected language: en
For Audio Segment 55.mp3
 A goon lives in a slum, i

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

For Audio Segment 111.mp3
এই ছাড়াও আমাদের যেহেতু ভারতীয় কালচার একটা আচ্ছিত শেখায়তে কেনাবিস্ত ইউস করে পিপলিউ নোমার বাববদেস কারুর রত সংস্যা হয় না যারা ইউস করে তারা নিজেদের মধ্যে থাকে বিকাজিতাটা। এছাড়া আরো প্রচিনেশা আছে বার খুক করেছে।

Detected language: bn


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


For Audio Segment 112.mp3
ক্রম ইউজে সরাহাত খুব একশর মধ্যাহত একজন দুজন ডাইপে কার নরমাল ফোসেল লেকটার স্যাম্পেল যদি আপনি নেন তারা যে ধরনের সাবসেন্স হেবিউজ করতে পারে সইই সেই জিনিসগুলি খানাকিরি।

Detected language: en
For Audio Segment 113.mp3
 It's nothing extra. It's nothing extra. It's nothing extra. As a lawyer, do you feel that a profession, comparing profession to profession, advocacy profession, alcohol, kimbo...

Detected language: bn


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


For Audio Segment 114.mp3
সার্স্টেন্সেস ইউজ কম্প্যাটিগ্রি বেশি কিংবা কম হয়। ইজ্জাক্ত বলতেও আর এখানের ক্ষেত্রে পশ্চিম মংলাও নয় কোলকাতা এরিয়া রাসপাষ্ট মেঙ্গ হতেই। আইটি সেক্টার ব্যাঙ্গেলরোধে যেটা কর্মে হতেই পেনেছেন।

 
  So could you please describe a typical day in the life of a lawyer? এবার সত্তার প্রধান দিকে আকল্যাওয়ান করেন যে কি হওয়া করতে কি ইনতি। ইসাবেজ এই রাফর আমাদের একটা লিস থাকবো। যে এভিডি একটা মামলা আছে করতে অনেকটি সকালেতে প্রথম আমাদের ইহচ্ছে যে কোনও কম্প্রিকেশন সারাইইস করেছে। এই বড় কমলিয়েশন্স এইগুলি হয় যাজছায় ঐদিংকে বসছেন। প্রথম হাসার। যাজছায় বসবেন না মুহার্ভোয়াস আছে নফটো অ্যাটেন্ডাসে এগুলির রেগগুলির একটা থাকে। এর পরাসে লিগেল কমটিকেশন্স। আমরা হয়তো কিছু স্ট্রেডিজি প্ল্যান করয়েছে। সাবমেশনস আছে কিছু লোপয়েন্স আছে সেকুল সাবমেশন করলেন কার তারপর জানা গেল সেগুলো অ্যাক্লিকিভাবে হচ্ছে না কার কোজ দেফেন্স অফ ইন্টারপ্রেটেশন। দ্যাথার প্রভ্লেম ভিচার রাইজেস একটি মোমেন্ট পিনিট ডিসাইড সামথিক অ্যাড দিমোমেন্ট কুই � তামাদের রিসেন্ট এখানা যেকম আছে যে কর্তেতে মামলা হয় চিক্যান্সিলেশন অফ ভেলের জন্য তামা

In [ ]:
import json
with open('result.json', 'w') as fp:
    json.dump(all_result, fp)

#Read the Result File

In [ ]:
result_file_path = '/content/audio/result.json'
f = open(result_file_path) 
data = json.load(f)

print(f"Keys are {data.keys()}")

Keys are dict_keys(['Advocate_1.mp3', 'Advocate_2.mp3'])


# Transcript

In [ ]:
data['Advocate_1.mp3']

"  Okay, so could you please describe me a typical day in the life of a lawyer and the aspects of your work that you find most rewarding in this profession.  See, for me, for any lawyer, a person comes to a lawyer, whenever he or she is in a problem, that problem can be...  Whatever it could be, the nature of the problem is different. But it can be civil in nature, it can be criminal in nature or it can be corporate in nature. So, obviously, the charge of fees is very low.  for doing our legal work. But the most rewarding part is when I give relief to a client and he comes and appreciates about, okay sir, thank you.  I was stuck with this job for a long time after coming to U.N. and got the job solved. So this is, fees is obviously, that is, you are practicing.  That is your income, that is your bread and butter. The fees, your bread and butter comes from the fees you get from your clients. But the reward is when the token of appreciation is the main reward of all. To me, I want to kno

In [ ]:
result['Advocate_2.mp3']

"  So could you please describe a typical day in the life of a lawyer? এবার সত্তার প্রধান দিকে আকল্যাওয়ান করেন যে কি হওয়া করতে কি ইনতি। ইসাবেজ এই রাফর আমাদের একটা লিস থাকবো। যে এভিডি একটা মামলা আছে করতে অনেকটি সকালেতে প্রথম আমাদের ইহচ্ছে যে কোনও কম্প্রিকেশন সারাইইস করেছে। এই বড় কমলিয়েশন্স এইগুলি হয় যাজছায় ঐদিংকে বসছেন। প্রথম হাসার। যাজছায় বসবেন না মুহার্ভোয়াস আছে নফটো অ্যাটেন্ডাসে এগুলির রেগগুলির একটা থাকে। এর পরাসে লিগেল কমটিকেশন্স। আমরা হয়তো কিছু স্ট্রেডিজি প্ল্যান করয়েছে। সাবমেশনস আছে কিছু লোপয়েন্স আছে সেকুল সাবমেশন করলেন কার তারপর জানা গেল সেগুলো অ্যাক্লিকিভাবে হচ্ছে না কার কোজ দেফেন্স অফ ইন্টারপ্রেটেশন। দ্যাথার প্রভ্লেম ভিচার রাইজেস একটি মোমেন্ট পিনিট ডিসাইড সামথিক অ্যাড দিমোমেন্ট কুই � তামাদের রিসেন্ট এখানা যেকম আছে যে কর্তেতে মামলা হয় চিক্যান্সিলেশন অফ ভেলের জন্য তামার স্র্যাটিস ছিল যে ফোন নাইনডিএটি মামলা হয়েছবানে বায়েলেন্স আগেন্স দ্য বয়। তুষ সেখানে কোনও আয়োর কোনও অ্যাডবেজ কর্নো রিপোর্ট বাট। যাদশায় ক্ষারকে এদিলেন যে না যেহেতু এরা মাঙি রিকাভারির একটা জায়গা আছে তো বিধাও উপানে রিকা

In [ ]:
import os
import glob

files = glob.glob('/content/audio/Advocate_1/*')
for f in files:
    os.remove(f)